### Pre-training with BERT

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import shutil
shutil.copytree("/content/drive/MyDrive/GitHub/exBERT/exBERT", "./exBERT")

'./exBERT'

In [4]:
!pip install transformers
!pip install tokenizers

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
     |████████████████████████████████| 596 kB 58.0 MB/s 
     |████████████████████████████████| 6.5 MB 47.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
!pip install boto3

     |████████████████████████████████| 132 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 8.6 MB 40.8 MB/s 
     |████████████████████████████████| 138 kB 38.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
!pip install datasets git+https://github.com/huggingface/transformers/
!pip install torch torchvision tensorflow
!pip install pytorch-pretrained-bert pandas

  Cloning https://github.com/huggingface/transformers/ to /tmp/pip-req-build-g4dkajd8
  Running command git clone -q https://github.com/huggingface/transformers/ /tmp/pip-req-build-g4dkajd8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 134 kB 49.2 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 
     |████████████████████████████████| 212 kB 56.4 MB/s 
     |████████████████████████████████| 127 kB 59.6 MB/s 
     |████████████████████████████████| 144 kB 37.8 MB/s 
     |████████████████████████████████| 271 kB 62.7 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3946315 sha256=e10aaf585072e8dfc689148219b22a51ce67f603db0b26abc7aad8e25553e7f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-5r0

In [8]:
!ls /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended

 added_tokens.json	   pubmed_config_ex_base_s3.json   tokenizer.json
 config.json		   pytorch_model.bin		   vocab.txt
'Copy of tokenizer.json'   special_tokens_map.json
 extended_vocab.txt	   tokenizer_config.json


Generate pre-training data. Please see https://github.com/google-research/bert for more details.

In [10]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForMaskedLM

In [11]:
updated_model = '/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended'
config = AutoConfig.from_pretrained(updated_model)

In [12]:
model = AutoModelForMaskedLM.from_pretrained(updated_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(updated_model)

Some weights of BertForMaskedLM were not initialized from the model checkpoint at /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
#Testing
from exBERT import BertTokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


Generating Training Data

In [14]:
#For this we need raw text
!python /content/drive/MyDrive/GitHub/exBERT/data_preprocess.py -voc /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended/extended_vocab.txt -ls 128 -dp /content/drive/MyDrive/GitHub/exBERT/data/exbert_train_data.txt -n_c 5 -rd 1 -sp /content/drive/MyDrive/GitHub/exBERT/data/exbert_train_data_output.pkl

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
processing data, thie might take some time
100% 5/5 [00:00<00:00, 52958.38it/s]
generating random sequence


In [15]:
!nvidia-smi

Tue Mar 29 06:46:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
device = 'cuda'
import torch, gc
import os
gc.collect()
torch.cuda.empty_cache()

In [17]:
!python /content/drive/MyDrive/GitHub/exBERT/Pretraining.py -e 1 -b 4 -sp /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_output -dv 0 -lr 1e-04 -str exBERT -config /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended/config.json /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended/pubmed_config_ex_base_s3.json -vocab /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended/extended_vocab.txt -pm_p /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended/pytorch_model.bin -dp /content/drive/MyDrive/GitHub/exBERT/data/exbert_train_data_output.pkl -ls 128 -p 1 -t_ex_only ""


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
epochs: 1
batchsize: 4
save_path: /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_output
device: [0]
learning_rate: 0.0001
strategy: exBERT
config: ['/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended/config.json', '/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended/pubmed_config_ex_base_s3.json']
vocab: /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended/extended_vocab.txt
pretrained_model_path: /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed_extended/pytorch_model.bin
pretrained_model_path_tf: None
datat_path: /content/drive/MyDrive/GitHub/exBERT/data/exbert_train_data_output.pkl
longest_sentence: 128
percentage: 1.0
sep: 1
warmup: -1
train_extension_only: False
training with GPU: [0]
Building PyTorch model from configuration: {
  "_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-full

**************** STOPPED HERE - NEED TO RE FORMAT NER / RE DATA****************

In [23]:
!python /content/drive/MyDrive/GitHub/exBERT/Finetuning_NER.py -e 1 -b 2 -sp /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/NER -dv 0 -lr 1e-04 -str exBERT -config /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed/config.json /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed/pubmed_config_ex_base_s3.json -vocab /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed/vocab.txt -pm_p /content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed/pytorch_model.bin -dp /content/drive/MyDrive/GitHub/exBERT/ner_sample_data/ -tln 6 7 8 9 10 11 12


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
{'epochs': 1, 'batchsize': 2, 'save_path': '/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/NER', 'device': [0], 'learning_rate': 0.0001, 'strategy': 'exBERT', 'config': ['/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed/config.json', '/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed/pubmed_config_ex_base_s3.json'], 'vocab': '/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed/vocab.txt', 'pretrained_model_path': '/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed/pytorch_model.bin', 'data_path': '/content/drive/MyDrive/GitHub/exBERT/ner_sample_data/', 'train_layer_number': ['6', '7', '8', '9', '10', '11', '12']}
[0]
load file: /content/drive/MyDrive/GitHub/exBERT/ner_sample_data/train.txt
seprate sentence
tokenization
error num:0
complete
load file: /content/drive/MyDrive/GitHub/exBERT/ner_sample_data/dev.txt
seprate sentence
tokenization

In [15]:
!python bert/create_pretraining_data.py --input_file=/content/drive/MyDrive/GitHub/exBERT/data/train_data.txt --output_file=/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/pubmed_uncased_train_data.tfrecord --vocab_file=/content/drive/MyDrive/GitHub/exBERT/PubMED_train_files/PubMed/vocab.txt --do_lower_case=True --max_seq_length=512 --max_predictions_per_seq=20 --masked_lm_prob=0.15 --random_seed=12345 --dupe_factor=5


Traceback (most recent call last):
  File "bert/create_pretraining_data.py", line 24, in <module>
    from bert import tokenization
ModuleNotFoundError: No module named 'bert'


https://medium.com/@manasmohanty/ncbi-bluebert-ncbi-bert-using-tensorflow-weights-with-huggingface-transformers-15a7ec27fc3d

We used the following code to train the BERT model. Please do not include init_checkpoint if you are pre-training from scratch. Please see https://github.com/google-research/bert for more details.

In [ ]:
python bert/run_pretraining.py \
  --input_file=pubmed_uncased_sentence_nltk.tfrecord \
  --output_dir=$BlueBERT_DIR \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=$BlueBERT_DIR/bert_config.json \
  --init_checkpoint=$BlueBERT_DIR/bert_model.ckpt \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20000 \
  --num_warmup_steps=10 \
  --learning_rate=2e-5